In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import faiss
import pickle

In [2]:
#data_target = pd.read_csv("../Masterskaya-2/data/train.csv", index_col=0)
data_base = pd.read_csv("../Masterskaya-2/data/base.csv", index_col=0)
data_validation = pd.read_csv("../Masterskaya-2/data/validation.csv", index_col=0)
data_validation_answer = pd.read_csv("../Masterskaya-2/data/validation_answer.csv", index_col=0)["Expected"]

In [3]:
with open('model.pkl', 'rb') as fid:
    model_regressor = pickle.load(fid)
    fid.close()

In [4]:
prediction = model_regressor.predict(data_validation)
model_regressor = None

In [5]:
data_validation = None

In [6]:
model = faiss.read_index("model.faiss")
model = faiss.index_cpu_to_all_gpus(model)
model.nprobe = 10

In [7]:
base_index = {k: v for k, v in enumerate(pd.DataFrame(data_base).index.to_list())}

In [16]:
vecs, idx = model.search(np.ascontiguousarray(prediction).astype('float32'), 1000)

In [17]:
acc = 0
for target, el in zip(data_validation_answer.values.tolist(), idx.tolist()):
    acc += int(target in [base_index[r] for r in el])
print(100 * acc / len(idx))

29.464


In [10]:
# acc = 0
# for target, el in zip(targets.values.tolist(), idx.tolist()):
#     acc += int(target in [base_index[r] for r in el])
# print(100 * acc / len(idx))